# LLVM Cauldron - Wuthering Bytes 2016-09-08

# Generating Python & Ruby bindings from C++

### Jonathan B Coe
### jbcoe@me.com

## https://github.com/jbcoe/C_API_generation
## https://github.com/jbcoe/clang_cpp_code_model

Write a C++ class out to a file in the current working directory

In [ ]:
outputfile = "Shape.h"

In [ ]:
%%file $outputfile
#include <cmath>
#include <stdexcept>
#include <string>

struct Shape
{
  virtual ~Shape()
  {
  }
  virtual double area() const = 0;
  virtual double perimeter() const = 0;
  virtual const char* name() const = 0;
} __attribute__((annotate("GENERATE_C_API")));

static const double pi = 3.14159265359;

class Circle : public Shape
{
  const double radius_;

public:
  double area() const override
  {
    return pi * radius_ * radius_;
  }

  double perimeter() const override
  {
    return 2 * pi * radius_;
  }

  const char* name() const override
  {
    return "Circle";
  }

  Circle(double radius) : radius_(radius)
  {
    if ( radius < 0 ) 
    { 
      std::string s = "Circle radius \"" + std::to_string(radius_) + "\" must be non-negative.";
      throw std::runtime_error(s);
    }
  }
};

Compile our header to check it's valid C++

In [ ]:
%%sh
clang++-3.8 -x c++ -fsyntax-only -std=c++14 Shape.h 

Read the code using libclang

In [ ]:
import clang.cindex

index = clang.cindex.Index.create()
translation_unit = index.parse(outputfile, ['-x', 'c++', '-std=c++14'])

In [ ]:
import asciitree

def node_children(node):
    return (c for c in node.get_children() if c.location.file.name == outputfile)

print asciitree.draw_tree(translation_unit.cursor,
  lambda n: [c for c in node_children(n)],
  lambda n: "%s (%s)" % (n.spelling or n.displayname, str(n.kind).split(".")[1]))


Turn the AST into some easy to manipulate Python classes

In [ ]:
import sys
sys.path.insert(0,'../externals/clang_cpp_code_model')
import cppmodel

In [ ]:
model = cppmodel.Model(translation_unit)

In [ ]:
[f.name for f in model.functions]

In [ ]:
[c.name for c in model.classes]

In [ ]:
shape_class = [c for c in model.classes if c.name=='Shape'][0]

In [ ]:
["{}::{}".format(shape_class.name,m.name) for m in shape_class.methods]

Look at the templates the generator uses

In [ ]:
%cat ../templates/json.tmpl

Run the code generator

In [ ]:
%%sh
export PYTHONPATH=../externals/clang_cpp_code_model:$PYTHONPATH
../generate.py Shape.h ../templates/ . 

See what it created

In [ ]:
%ls

Build some bindings with the generated code.

In [ ]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)
set(CMAKE_CXX_COMPILER /usr/local/bin/clang++-3.8)
add_library(Shape_c SHARED Shape_c.cpp)

In [ ]:
%%sh
cmake . 
cmake --build .

In [ ]:
%%python2
import Shape
Shape.Config.set_library_path("/Users/jon/DEV/C_APICodeGen/demos/")
c = Shape.Circle(8)

print "A {} with radius {} has area {}".format(c.name(), 8, c.area())

In [ ]:
%%script pypy
import Shape
Shape.Config.set_library_path("/Users/jon/DEV/C_APICodeGen/demos/")
c = Shape.Circle(8)

print "A {} with radius {} has area {}".format(c.name(), 8, c.area())

In [ ]:
%%ruby
load "Shape.rb"
c = Circle.new(8)

puts("A #{c.name()} with radius #{8} has area #{c.area()}")